In [1]:
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Data Preparation

In [5]:
import pandas as pd

# --- 1. Load your data ---
# If your data is in a CSV, use this:
df = pd.read_csv('dataset.csv')  # Use sep='\t' if your file is tab-separated
print("Columns in DataFrame:", df.columns.tolist())
# If you are pasting the data directly, you can use StringIO (uncomment below):
# from io import StringIO
# data = """ID\tSystemCodeNumber\tCapacity\tLatitude\tLongitude\tOccupancy\tVehicleType\tTrafficConditionNearby\tQueueLength\tIsSpecialDay\tLastUpdatedDate\tLastUpdatedTime
# 0\tBHMBCCMKT01\t577\t26.14453614\t91.73617216\t61\tcar\tlow\t1\t0\t04-10-2016\t7:59:00
# ... (rest of your data here) ...
# """
# df = pd.read_csv(StringIO(data), sep='\t')

# --- 2. Strip whitespace from all column names ---
df.columns = df.columns.str.strip()

# --- 3. Map TrafficConditionNearby to numeric ---
traffic_map = {'low': 0.0, 'average': 1.0, 'high': 2.0}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].replace(traffic_map)
df['TrafficConditionNearby'] = pd.to_numeric(df['TrafficConditionNearby'], errors='coerce').fillna(0.0).astype(float)

# --- 4. Map VehicleType to numeric weight ---
veh_type_map = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.25}
df['veh_type_weight'] = df['VehicleType'].replace(veh_type_map).fillna(1.0).astype(float)

# --- 5. Combine date and time into a single timestamp string ---
df['Timestamp'] = df['LastUpdatedDate'].astype(str).str.strip() + ' ' + df['LastUpdatedTime'].astype(str).str.strip()

# --- 6. Ensure all numeric columns are correct type ---
int_cols = ['Capacity', 'Occupancy', 'QueueLength', 'IsSpecialDay']
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

float_cols = ['Latitude', 'Longitude', 'TrafficConditionNearby', 'veh_type_weight']
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype(float)

# --- 7. Save the cleaned DataFrame ---
df.to_csv('parking_stream_full.csv', index=False, encoding='utf-8')

# --- 8. Print to verify ---
print(df.dtypes)
print(df.head())


Columns in DataFrame: ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
ID                          int64
SystemCodeNumber           object
Capacity                    int64
Latitude                  float64
Longitude                 float64
Occupancy                   int64
VehicleType                object
TrafficConditionNearby    float64
QueueLength                 int64
IsSpecialDay                int64
LastUpdatedDate            object
LastUpdatedTime            object
veh_type_weight           float64
Timestamp                  object
dtype: object
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172    

/tmp/ipython-input-5-1357878986.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['TrafficConditionNearby'] = df['TrafficConditionNearby'].replace(traffic_map)
/tmp/ipython-input-5-1357878986.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['veh_type_weight'] = df['VehicleType'].replace(veh_type_map).fillna(1.0).astype(float)


Model 1: Baseline Linear Pricing Logic: Price increases linearly with occupancy rate.

Price t + 1 = Price t + α ⋅ ( Occupancy Capacity ) Price t+1 =Price t +α⋅( Capacity Occupancy )

In [7]:
BASE_PRICE = 10
alpha = 5  # Sensitivity coefficient, tune as needed

df['BaselinePrice'] = BASE_PRICE
for idx in range(1, len(df)):
    if df.loc[idx, 'ID'] == df.loc[idx-1, 'ID']:
        occ_rate = df.loc[idx-1, 'Occupancy'] / max(df.loc[idx-1, 'Capacity'], 1)
        df.loc[idx, 'BaselinePrice'] = df.loc[idx-1, 'BaselinePrice'] + alpha * occ_rate
    else:
        df.loc[idx, 'BaselinePrice'] = BASE_PRICE  # Reset for new lot

# Optionally clip to a reasonable range
df['BaselinePrice'] = df['BaselinePrice'].clip(lower=5, upper=30)

print(df[['ID', 'Timestamp', 'Occupancy', 'Capacity', 'BaselinePrice']].head(10))


   ID            Timestamp  Occupancy  Capacity  BaselinePrice
0   0  04-10-2016 07:59:00         61       577             10
1   1  04-10-2016 08:25:00         64       577             10
2   2  04-10-2016 08:59:00         80       577             10
3   3  04-10-2016 09:32:00        107       577             10
4   4  04-10-2016 09:59:00        150       577             10
5   5  04-10-2016 10:26:00        177       577             10
6   6  04-10-2016 10:59:00        219       577             10
7   7  04-10-2016 11:25:00        247       577             10
8   8  04-10-2016 11:59:00        259       577             10
9   9  04-10-2016 12:29:00        266       577             10


Model 2: Demand-Based Dynamic Pricing
Demand Function:

Demand
=
α
⋅
(
Occupancy
Capacity
)
+
β
⋅
QueueLength
−
γ
⋅
TrafficConditionNearby
+
δ
⋅
IsSpecialDay
+
ε
⋅
veh_type_weight
Demand=α⋅(
Capacity
Occupancy
 )+β⋅QueueLength−γ⋅TrafficConditionNearby+δ⋅IsSpecialDay+ε⋅veh_type_weight
Dynamic Price:

DynamicPrice
t
=
BasePrice
⋅
(
1
+
λ
⋅
NormalizedDemand
)
DynamicPrice
t
 =BasePrice⋅(1+λ⋅NormalizedDemand)

In [8]:
# Hyperparameters (tune as needed)
alpha = 2.0
beta = 0.5
gamma = 0.3
delta = 1.0
epsilon = 0.7
lmbda = 0.6

# Calculate raw demand
df['RawDemand'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] -
    gamma * df['TrafficConditionNearby'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['veh_type_weight']
)

# Normalize demand (z-score or min-max, here min-max)
demand_min = df['RawDemand'].min()
demand_max = df['RawDemand'].max()
df['NormDemand'] = (df['RawDemand'] - demand_min) / (demand_max - demand_min + 1e-6)

# Calculate dynamic price
df['DynamicPrice'] = BASE_PRICE * (1 + lmbda * df['NormDemand'])
df['DynamicPrice'] = df['DynamicPrice'].clip(lower=5, upper=30)  # Keep prices reasonable

print(df[['ID', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficConditionNearby', 'DynamicPrice']].head(10))


   ID            Timestamp  Occupancy  QueueLength  TrafficConditionNearby  \
0   0  04-10-2016 07:59:00         61            1                     0.0   
1   1  04-10-2016 08:25:00         64            1                     0.0   
2   2  04-10-2016 08:59:00         80            2                     0.0   
3   3  04-10-2016 09:32:00        107            2                     0.0   
4   4  04-10-2016 09:59:00        150            2                     0.0   
5   5  04-10-2016 10:26:00        177            3                     0.0   
6   6  04-10-2016 10:59:00        219            6                     2.0   
7   7  04-10-2016 11:25:00        247            5                     1.0   
8   8  04-10-2016 11:59:00        259            5                     1.0   
9   9  04-10-2016 12:29:00        266            8                     2.0   

   DynamicPrice  
0     10.614608  
1     10.620854  
2     10.954467  
3     11.010676  
4     10.889982  
5     11.456708  
6     12.294905

Real-Time Visualization with Bokeh

In [10]:
!pip install bokeh --quiet

from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
import pandas as pd
import time

output_notebook()


In [11]:
# For demo: select a single parking lot (e.g., first unique ID)
lot_id = df['ID'].iloc[0]
lot_df = df[df['ID'] == lot_id].copy().sort_values('Timestamp')

# Convert Timestamp to datetime
lot_df['Timestamp'] = pd.to_datetime(lot_df['Timestamp'])

# Initial data for the plot (first 10 points)
source = ColumnDataSource(data={
    'Timestamp': lot_df['Timestamp'].iloc[:10],
    'BaselinePrice': lot_df['BaselinePrice'].iloc[:10],
    'DynamicPrice': lot_df['DynamicPrice'].iloc[:10],
})


In [14]:
p = figure(title=f"Real-Time Pricing for Lot {lot_id}",
           x_axis_type='datetime',
           width=800, height=400)
p.line('Timestamp', 'BaselinePrice', source=source, color='blue', legend_label='BaselinePrice', line_width=2)
p.line('Timestamp', 'DynamicPrice', source=source, color='orange', legend_label='DynamicPrice', line_width=2)
p.legend.location = 'top_left'
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price ($)'
p.add_tools(HoverTool(
    tooltips=[
        ("Time", "@Timestamp{%F %T}"),
        ("Baseline", "@BaselinePrice{$0.00}"),
        ("Dynamic", "@DynamicPrice{$0.00}")
    ],
    formatters={'@Timestamp': 'datetime'},
    mode='vline'
))
handle = show(p, notebook_handle=True)


In [15]:
# This will append new points to the plot every 0.5 seconds
for i in range(10, len(lot_df)):
    new_data = {
        'Timestamp': [lot_df['Timestamp'].iloc[i]],
        'BaselinePrice': [lot_df['BaselinePrice'].iloc[i]],
        'DynamicPrice': [lot_df['DynamicPrice'].iloc[i]],
    }
    source.stream(new_data, rollover=60)  # keeps last 60 points
    push_notebook(handle=handle)
    time.sleep(0.5)  # simulate real-time arrival


Pathway Streaming

In [16]:
import pathway as pw

class ParkingSchema(pw.Schema):
    ID: int
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str
    veh_type_weight: float

data = pw.demo.replay_csv(
    "parking_stream_full.csv",
    schema=ParkingSchema,
    input_rate=1000
)

pw.run()


Output()